In [2]:
import sys
sys.path.append('../')

In [3]:
######################## Load modules ###################################
%matplotlib inline
import sys
import importlib
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from scipy import stats

from plants_and_TCR.analysis_parameters import directory_information
from plants_and_TCR.analysis_parameters import params
from plants_and_TCR.analysis_parameters import get_CMIP_info
from plants_and_TCR.analyze_data import grab_cmip_dataset
from plants_and_TCR.analyze_data import moving_average as movingaverage

####################### Set up directory structure ######################
PATH_REGRIDDED_DATA = directory_information.DIR_PROCESSED_DATA
PATH_FIGURES = directory_information.DIR_OUTPUT_FIGURES

In [4]:
DEFAULT_VARNAME = params.DEFAULT_VARNAME
runnames_all = params.RUNNAMES_ALL

dir_CMIPdicts = directory_information.DIR_DATA_DICTIONARIES
CMIP_DICT = pickle.load(open(dir_CMIPdicts+'cmip_dict.pickle',"rb"))

### Run stats

In [10]:
def get_baseline_values(modelname, varname=DEFAULT_VARNAME, runname='piControl',cmip_dict=CMIP_DICT, area_avg=None):
    pi_control_ds = grab_cmip_dataset.grab_cmip_dataset(cmip_dict,
                                                        modelname,
                                                        runname,
                                                        varname)
    if pi_control_ds is not None:
        pi_control = pi_control_ds[DEFAULT_VARNAME]
        cell_area = CMIP_DICT[modelname+'_'+'areacella']['areacella']
        area_weights = cell_area/cell_area.sum(dim=['lat','lon']).values
        
        land_frac = CMIP_DICT[modelname +'_sftlf']['sftlf'].reindex_like(cell_area, method='nearest', tolerance=0.001) # in values of 0 to 100
        glac_frac = CMIP_DICT[modelname +'_sftgif']['sftgif'].reindex_like(cell_area, method='nearest', tolerance=0.001)
        
        land_area = cell_area*(land_frac/100)*(1-(glac_frac/100))
        land_area_weights = land_area/land_area.sum(dim=['lat','lon']).values
        
        if area_avg =='land':
            pi_control_weighted = pi_control*land_area_weights
        elif ((area_avg == None) or (area_avg == 'global')):
            pi_control_weighted = pi_control*area_weights
        pi_control_global = pi_control_weighted.sum(dim=['lat','lon'])
        baseline = pi_control_global.mean(dim='time').values
        baseline_avgs = movingaverage.movingaverage(pi_control_global, 20*12) - baseline
    else:
        baseline_avgs=None
        baseline_avgs_years=None
        
    return baseline_avgs

In [5]:
from plants_and_TCR.analyze_data import make_tcr_dataset

In [6]:
DIR_TCR_DICT = directory_information.DIR_TCR_DICT
TCR_DICT = pickle.load(open(DIR_TCR_DICT+'TCR_dict.pickle','rb'))

In [7]:
modelnames = get_CMIP_info.get_modelnames_short('CMIP5and6')
colnames = ['Global_2xCO2_raw','Global_4xCO2_raw','Land_2xCO2_raw','Land_4xCO2_raw',
           'Global_2xCO2_detrend','Global_4xCO2_detrend','Land_2xCO2_detrend','Land_4xCO2_detrend',
           'Global_PI_SD','Land_PI_SD']
stats_table = pd.DataFrame(index=modelnames,columns=colnames)
AVERAGE_TYPES = ['global','global','land','land']
END_YRS=[70,130,70,130]

In [8]:
def calculate_p_val(baseline, signal):
    meanval = np.nanmean(baseline)
    stddev = np.std(baseline)
    z_score = (signal - meanval) / stddev
    pval = stats.norm.sf(z_score)
    return pval

In [11]:
for i in range(0,4):
    colname_raw = colnames[i]
    colname_detrend = colnames[i+4]
    AVERAGE_TYPE = AVERAGE_TYPES[i]
    END_YR = END_YRS[i]

    # Calculate TCRS
    global_tcrs = make_tcr_dataset.make_tcr_dataset(TCR_DICT, end_yr = END_YR,
                                                average_type=AVERAGE_TYPE, varname='tas')
    tcrs_phys = global_tcrs['TOT-RAD']
    
    # Calculate p-value for each model
    for m in range(0, len(modelnames)):
        modelname = modelnames[m]  
        print('-------'+modelname+'-------')

        #------------ Get distribution of PI control --------------
        baseline_avgs = get_baseline_values(modelname=modelname, varname='tas',
                                            runname='piControl',cmip_dict=CMIP_DICT, area_avg=AVERAGE_TYPE)
        if baseline_avgs is not None:
            times = np.arange(0,len(baseline_avgs))
            [slope, intercept, _, _, _] = stats.linregress(times, baseline_avgs)
            linearfit = slope*(times)+intercept
            detrended_baseline_avgs = baseline_avgs - linearfit
            if AVERAGE_TYPE=='global':
                stats_table['Global_PI_SD'][modelname] = np.std(detrended_baseline_avgs)
            elif AVERAGE_TYPE=='land':
                stats_table['Land_PI_SD'][modelname] = np.std(detrended_baseline_avgs)

            #----------- Get TCR--------------
            warming_phys = tcrs_phys[modelname]

            #---------- Check statistics-------
            z_thresh = stats.norm.ppf(0.95, loc =0, scale = 1)

            # For original baseline data
            pval = calculate_p_val(baseline_avgs, warming_phys)
            stats_table[colname_raw][modelname] = pval
            
            if pval <= 0.05:
                print('YES: '+ str(pval))
            else:
                print('NO: '+ str(pval))

            # For detrended baseline data
            pval = (calculate_p_val(detrended_baseline_avgs, warming_phys))
            stats_table[colname_detrend][modelname] = pval
            if pval <= 0.05:
                print('YES: '+ str(pval))
            else:
                print('NO: '+ str(pval))
        else:
            print('No Baseline Data')
    

-------bcc-csm1-1-------
NO: 0.15907705520070226
NO: 0.15454214686321316
-------CanESM2-------
YES: 9.84912178901045e-06
YES: 5.057544150579137e-08
-------CESM1-BGC-------
YES: 0.000355900858797186
YES: 0.00023995371720954867
-------GFDL-ESM2M-------
NO: 0.4995522754504432
NO: 0.4908252844292317
-------HadGEM2-ES-------
YES: 6.483773099835045e-10
YES: 1.0205782161607176e-09
-------IPSL-CM5A-LR-------
NO: 0.2338035943784919
NO: 0.22408086134642186
-------NorESM1-ME-------
YES: 1.4724970835963208e-09
YES: 2.0358213858733497e-09
-------MPI-ESM-LR-------
YES: 0.03443803431543669
YES: 0.03242957579231493
-------CNRM-ESM2-1-------
NO: 0.3596092348725637
NO: 0.36129004826604083
-------BCC-CSM2-MR-------
NO: 0.3423957845766086
NO: 0.3039844146753997
-------CanESM5-------
YES: 0.021296184885040124
YES: 0.014985322560091021
-------CESM2-------
YES: 0.0032971416176076893
YES: 1.6789700060374598e-06
-------GISS-E2-1-G-------
YES: 0.00012972847110105434
YES: 4.091341433571287e-05
-------UKESM1-0-LL

In [12]:
def highlight_true(s):
    is_true = (s <= 0.05)
    return ['background-color: lemonchiffon' if v else '' for v in is_true]

In [13]:
cmip_model_order = ['bcc-csm1-1','CanESM2','CESM1-BGC','GFDL-ESM2M',
                    'HadGEM2-ES','IPSL-CM5A-LR','MPI-ESM-LR','NorESM1-ME',
                    'ACCESS-ESM1-5','BCC-CSM2-MR','CanESM5','CESM2',
                    'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','IPSL-CM6A-LR',
                    'MIROC-ES2L','MPI-ESM1-2-LR','NorESM2-LM','UKESM1-0-LL']
colnames_reordered = ['Global_PI_SD','Global_2xCO2_detrend','Global_4xCO2_detrend',
                      'Land_PI_SD','Land_2xCO2_detrend','Land_4xCO2_detrend',
                      'Global_2xCO2_raw','Global_4xCO2_raw',
                      'Land_2xCO2_raw','Land_4xCO2_raw']

stats_table_ordered = stats_table.reindex(cmip_model_order)
stats_table_ordered=stats_table_ordered.reindex(columns=colnames_reordered)

In [14]:
stats_table_ordered.style.apply(highlight_true)

,Global_PI_SD,Global_2xCO2_detrend,Global_4xCO2_detrend,Land_PI_SD,Land_2xCO2_detrend,Land_4xCO2_detrend,Global_2xCO2_raw,Global_4xCO2_raw,Land_2xCO2_raw,Land_4xCO2_raw
bcc-csm1-1,0.059385,0.154542,0.000310,0.047290,0.001113,0.000000,0.159077,0.000394,0.001224,0.000000
CanESM2,0.034993,0.000000,0.000000,0.045553,0.000000,0.000000,0.000010,0.000000,0.000000,0.000000
CESM1-BGC,0.032918,0.000240,0.000000,0.044946,0.000001,0.000000,0.000356,0.000000,0.000001,0.000000
GFDL-ESM2M,0.046704,0.490825,0.164302,0.064194,0.941374,0.632751,0.499552,0.169961,0.944177,0.643040
HadGEM2-ES,0.084603,0.000000,0.000000,0.112919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IPSL-CM5A-LR,0.043707,0.224081,0.000000,0.060844,0.005171,0.000000,0.233804,0.000000,0.005121,0.000000
MPI-ESM-LR,0.039579,0.032430,0.000000,0.061371,0.000020,0.000000,0.034438,0.000000,0.000026,0.000000
NorESM1-ME,0.024438,0.000000,0.000000,0.016268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACCESS-ESM1-5,0.054376,0.001807,0.000000,0.075494,0.001472,0.000000,0.002962,0.000000,0.001617,0.000000
BCC-CSM2-MR,0.141633,0.303984,0.110737,0.120554,0.172375,0.005560,0.342396,0.162590,0.225268,0.020010


In [15]:
stats_table_ordered.to_csv(PATH_FIGURES+'Table_S4.csv', index=True)